In [1]:
!pip install transformers torch torchvision einops timm peft sentencepiece

In [1]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

import pandas as pd

In [11]:
# Set up the model and tokenizer
model_path = 'h2oai/h2ovl-mississippi-800m'
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
#config.llm_config._attn_implementation = 'flash_attention_2'
config.vision_config.use_flash_attn = False
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    config=config,
    low_cpu_mem_usage=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=128, do_sample=True)

In [12]:
!ls

data.csv  images  sample_data


In [13]:
!ls images

doc_000016.png	doc_000097.png	doc_000279.png	doc_000402.png	doc_000551.png	doc_000674.png
doc_000045.png	doc_000101.png	doc_000281.png	doc_000485.png	doc_000558.png	doc_000745.png
doc_000070.png	doc_000130.png	doc_000297.png	doc_000509.png	doc_000584.png	doc_000802.png
doc_000076.png	doc_000175.png	doc_000349.png	doc_000513.png	doc_000596.png	doc_000832.png
doc_000089.png	doc_000248.png	doc_000363.png	doc_000550.png	doc_000655.png	doc_000864.png


In [14]:
df = pd.read_csv('data.csv')
print(df.shape)
df.head()

(30, 2)


,image,class
0,images/doc_000832.png,scientific_publication
1,images/doc_000076.png,email
2,images/doc_000584.png,scientific_publication
3,images/doc_000101.png,resume
4,images/doc_000279.png,email


In [19]:
%%time

sample_idx = 0
display(df.iloc[sample_idx,:])
image_file = df.loc[sample_idx,'image']

question = "<image>\nExtract the type of the image, categorizing it as 'email', 'resume' or 'scientific_publication'. Type:"

response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
print('response >> ', response)
print('history >> ', history)

,0
image,images/doc_000832.png
class,scientific_publication


response >>  scientific_publication
history >>  [("<image>\nExtract the type of the image, categorizing it as 'email', 'resume' or 'scientific_publication'. Type:", 'scientific_publication')]
CPU times: user 2.75 s, sys: 2.45 ms, total: 2.75 s
Wall time: 3.55 s


In [24]:
%%time

question = "<image>\nExtract the type of the image, categorizing it as 'email', 'resume' or 'scientific_publication'. Type:"

answer = []

for _, row in df.iterrows():
  image_file = row['image']
  response, history = model.chat(tokenizer, image_file, question, generation_config, history=None, return_history=True)
  print(image_file,  ' - ', row['class'], ' : ', response)
  answer.append(response)

question_1 = question
answer_1 = answer
print('----- DONE -----')

images/doc_000832.png  -  scientific_publication  :  scientific_publication
images/doc_000076.png  -  email  :  email
images/doc_000584.png  -  scientific_publication  :  scientific_publication
images/doc_000101.png  -  resume  :  email
images/doc_000279.png  -  email  :  email
images/doc_000248.png  -  resume  :  email
images/doc_000802.png  -  resume  :  email
images/doc_000089.png  -  scientific_publication  :  scientific_publication
images/doc_000097.png  -  resume  :  resume
images/doc_000509.png  -  scientific_publication  :  scientific_publication
images/doc_000550.png  -  email  :  email
images/doc_000349.png  -  scientific_publication  :  This image is a PDF document related to research conducted by the University of Texas at Southwest Medical Center and the University of Tennessee, Memphis. It is categorized under 'Publications' and 'Publications' of the journal Lonely Planet and 'Publication' of the University of Tennessee.
images/doc_000596.png  -  email  :  email
images/do

In [25]:
question_1

"<image>\nExtract the type of the image, categorizing it as 'email', 'resume' or 'scientific_publication'. Type:"

In [26]:
len(answer_1)

30

In [28]:
df[question_1] = answer_1
df

,image,class,"<image>\nExtract the type of the image, categorizing it as 'email', 'resume' or 'scientific_publication'. Type:"
0,images/doc_000832.png,scientific_publication,scientific_publication
1,images/doc_000076.png,email,email
2,images/doc_000584.png,scientific_publication,scientific_publication
3,images/doc_000101.png,resume,email
4,images/doc_000279.png,email,email
5,images/doc_000248.png,resume,email
6,images/doc_000802.png,resume,email
7,images/doc_000089.png,scientific_publication,scientific_publication
8,images/doc_000097.png,resume,resume
9,images/doc_000509.png,scientific_publication,scientific_publication


In [29]:
df.to_csv('VLM_result.csv',index=False)